# How To: Neural ODE

In the Neural ODE case, the whole RHS of each equation is replaced with a network.  So the Lotka-Voltera system,

$$\frac{\mathrm{d} \text{prey}}{\mathrm{d} t} = \alpha \cdot \text{prey} - \beta \cdot \text{prey} \cdot \text{predator}$$

$$\frac{\mathrm{d} \text{predator}}{\mathrm{d} t} = \gamma \cdot \text{prey} \cdot \text{predator} - \delta \cdot \text{predator}$$

simply becomes,

$$\frac{\mathrm{d} \text{prey}}{\mathrm{d} t} = \text{NN}(\text{prey}, \text{predator})[0]$$

$$\frac{\mathrm{d} \text{predator}}{\mathrm{d} t} = \text{NN}(\text{prey}, \text{predator})[1]$$

In the SBML model the species ``prey`` and ``predator`` are defined as these will be used as inputs to the neural network.  Also two parameters $\alpha$ and $\delta$ are defined.  These will be mapped to neural network outputs in the PEtab files.

## Defining the network architecture

In [38]:
from petab_sciml.standard.nn_model import Input, NNModel, NNModelStandard
import torch
from torch import nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(2, 5)
        self.layer2 = torch.nn.Linear(5, 5)
        self.layer3 = torch.nn.Linear(5, 1)

    def forward(self, net_input):
        x = self.layer1(net_input)
        x = F.sigmoid(x)
        x = self.layer2(x)
        x = F.sigmoid(x)
        x = self.layer3(x)
        return x

net1 = NeuralNetwork()
nn_model1 = NNModel.from_pytorch_module(
    module=net1, nn_model_id="net1", inputs=[Input(input_id="input0")]
)
NNModelStandard.save_data(
    data=nn_model1, filename="net1.yaml"
)

The network architecture in this example is kep simple for demonstration purposes.  Refer to the page on supported layers and activation functions for more inspiration, but not that PEtab SciML and its importers currently only support networks with vector outputs. 

## Generating the PEtab files

**TODO** The PEtab SciML python package provides a utility function to generate the model and PEtab files for a neural ODE case.  The names of the species in the ODE system are required, along with the measurements, network YAML and any array input files. 

In [41]:
# from petab_sciml.problem_utility.neural_ode import generate_neural_ode_problem
# generate_neural_ode_problem(["prey", "predator"], model_filename="lv.xml")

## Loading the PEtab problem

In [ ]:
from amici.petab import import_petab_problem
from amici.jax import (
    JAXProblem,
    run_simulations,
)
from petab.v2 import Problem

# Create the PEtab problem
petab_problem = Problem.from_yaml("problem.yaml")

# Create AMICI model for the petab problem
jax_model = import_petab_problem(
    petab_problem,
    model_output_dir="model",
    compile_=True,
    jax=True
)

# Create the JAXProblem - wrapper for the AMICI model
jax_problem = JAXProblem(jax_model, petab_problem)

The hybridization and mapping tables show us how the neural network inputs and outputs are mapped to the model.

In [8]:
jax_problem._petab_problem.hybridization_df

,targetValue
targetId,
net1_input1,prey
net1_input2,predator
alpha,net1_output1
delta,net1_output2


In [9]:
petab_problem.mapping_df

,modelEntityId
petabEntityId,
net1_input1,net1.inputs[0][0]
net1_input2,net1.inputs[0][1]
net1_output1,net1.outputs[0][0]
net1_output2,net1.outputs[0][1]
net1_ps,net1.parameters


The inputs to the neural network are given by the ``prey`` and ``predator`` species.  $\alpha$, the ``prey`` concentration over time as defined by the SBML model, is given by the first output of the network. $\delta$, the ``predator`` concentration over time, is given by the second output of the network.

It is also worth showing that the parameter table only has the network parameters defined in it. Unlike previous examples, there are no other parameters to be estimated in the problem. We will optimise the network parameters and then the outputs of the network will give us the solution to our ODEs.

In [10]:
petab_problem.parameter_df

,parameterScale,lowerBound,upperBound,nominalValue,estimate
parameterId,,,,,
net1_ps,lin,-inf,inf,NaN,1
